# SGLang Frontend Language

SGLang frontend language can be used to define simple and easy prompts in a convenient, structured way.

## Launch A Server

Launch the server in your terminal and wait for it to initialize.

In [1]:
import requests
import os

from sglang import assistant_begin, assistant_end
from sglang import assistant, function, gen, system, user
from sglang import image
from sglang import RuntimeEndpoint, set_default_backend
from sglang.srt.utils import load_image
from sglang.test.test_utils import is_in_ci
from sglang.utils import print_highlight, terminate_process, wait_for_server

if is_in_ci():
    from patch import launch_server_cmd
else:
    from sglang.utils import launch_server_cmd


server_process, port = launch_server_cmd(
    "python -m sglang.launch_server --model-path Qwen/Qwen2.5-7B-Instruct --host 0.0.0.0"
)

wait_for_server(f"http://localhost:{port}")
print(f"Server started on http://localhost:{port}")

[2025-05-18 11:49:01] server_args=ServerArgs(model_path='Qwen/Qwen2.5-7B-Instruct', tokenizer_path='Qwen/Qwen2.5-7B-Instruct', tokenizer_mode='auto', skip_tokenizer_init=False, load_format='auto', trust_remote_code=False, dtype='auto', kv_cache_dtype='auto', quantization=None, quantization_param_path=None, context_length=None, device='cuda', served_model_name='Qwen/Qwen2.5-7B-Instruct', chat_template=None, completion_template=None, is_embedding=False, enable_multimodal=None, revision=None, host='0.0.0.0', port=31814, mem_fraction_static=0.88, max_running_requests=200, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False, random_seed=388936487, constrained_json_whitespace_pattern=None, watchdog_timeout=300, dist_timeout=None, download_dir=None, base_gpu_id=0, gpu_id_step=1, log_level='info', 

[2025-05-18 11:49:09] Attention backend not set. Use fa3 backend by default.
[2025-05-18 11:49:09] Init torch distributed begin.
[2025-05-18 11:49:09] Init torch distributed ends. mem usage=0.00 GB
[2025-05-18 11:49:09] Load weight begin. avail mem=78.60 GB


[2025-05-18 11:49:10] Using model weights format ['*.safetensors']


Loading safetensors checkpoint shards:   0% Completed | 0/4 [00:00<?, ?it/s]


Loading safetensors checkpoint shards:  25% Completed | 1/4 [00:01<00:04,  1.55s/it]


...

[2025-05-18 11:49:18] INFO:     Started server process [1055583]
[2025-05-18 11:49:18] INFO:     Waiting for application startup.
[2025-05-18 11:49:18] INFO:     Application startup complete.
[2025-05-18 11:49:18] INFO:     Uvicorn running on http://0.0.0.0:31814 (Press CTRL+C to quit)


[2025-05-18 11:49:19] INFO:     127.0.0.1:34296 - "GET /v1/models HTTP/1.1" 200 OK


[2025-05-18 11:49:19] INFO:     127.0.0.1:34302 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-05-18 11:49:19] Prefill batch. #new-seq: 1, #new-token: 6, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-05-18 11:49:21] INFO:     127.0.0.1:34312 - "POST /generate HTTP/1.1" 200 OK
[2025-05-18 11:49:21] The server is fired up and ready to roll!


Server started on http://localhost:31814


Set the default backend. Note: Besides the local server, you may use also `OpenAI` or other API endpoints.

In [2]:
set_default_backend(RuntimeEndpoint(f"http://localhost:{port}"))

[2025-05-18 11:49:24] INFO:     127.0.0.1:47516 - "GET /get_model_info HTTP/1.1" 200 OK


## Basic Usage

The most simple way of using SGLang frontend language is a simple question answer dialog between a user and an assistant.

In [3]:
@function
def basic_qa(s, question):
    s += system(f"You are a helpful assistant than can answer questions.")
    s += user(question)
    s += assistant(gen("answer", max_tokens=512))

In [4]:
state = basic_qa("List 3 countries and their capitals.")
print_highlight(state["answer"])

[2025-05-18 11:49:24] Prefill batch. #new-seq: 1, #new-token: 31, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-05-18 11:49:24] Decode batch. #running-req: 1, #token: 64, token usage: 0.00, cuda graph: False, gen throughput (token/s): 6.11, #queue-req: 0
[2025-05-18 11:49:24] INFO:     127.0.0.1:47528 - "POST /generate HTTP/1.1" 200 OK


## Multi-turn Dialog

SGLang frontend language can also be used to define multi-turn dialogs.

In [5]:
@function
def multi_turn_qa(s):
    s += system(f"You are a helpful assistant than can answer questions.")
    s += user("Please give me a list of 3 countries and their capitals.")
    s += assistant(gen("first_answer", max_tokens=512))
    s += user("Please give me another list of 3 countries and their capitals.")
    s += assistant(gen("second_answer", max_tokens=512))
    return s


state = multi_turn_qa()
print_highlight(state["first_answer"])
print_highlight(state["second_answer"])

[2025-05-18 11:49:24] Prefill batch. #new-seq: 1, #new-token: 18, #cached-token: 18, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-05-18 11:49:25] Decode batch. #running-req: 1, #token: 1, token usage: 0.00, cuda graph: False, gen throughput (token/s): 62.45, #queue-req: 0
[2025-05-18 11:49:25] INFO:     127.0.0.1:47530 - "POST /generate HTTP/1.1" 200 OK


[2025-05-18 11:49:25] Prefill batch. #new-seq: 1, #new-token: 23, #cached-token: 74, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-05-18 11:49:25] Decode batch. #running-req: 1, #token: 1, token usage: 0.00, cuda graph: False, gen throughput (token/s): 63.42, #queue-req: 0
[2025-05-18 11:49:25] INFO:     127.0.0.1:47542 - "POST /generate HTTP/1.1" 200 OK


## Control flow

You may use any Python code within the function to define more complex control flows.

In [6]:
@function
def tool_use(s, question):
    s += assistant(
        "To answer this question: "
        + question
        + ". I need to use a "
        + gen("tool", choices=["calculator", "search engine"])
        + ". "
    )

    if s["tool"] == "calculator":
        s += assistant("The math expression is: " + gen("expression"))
    elif s["tool"] == "search engine":
        s += assistant("The key word to search is: " + gen("word"))


state = tool_use("What is 2 * 2?")
print_highlight(state["tool"])
print_highlight(state["expression"])

[2025-05-18 11:49:25] Prefill batch. #new-seq: 1, #new-token: 25, #cached-token: 8, token usage: 0.00, #running-req: 0, #queue-req: 0
[2025-05-18 11:49:26] INFO:     127.0.0.1:47558 - "POST /generate HTTP/1.1" 200 OK
[2025-05-18 11:49:26] Prefill batch. #new-seq: 1, #new-token: 2, #cached-token: 31, token usage: 0.00, #running-req: 0, #queue-req: 0
[2025-05-18 11:49:26] Prefill batch. #new-seq: 1, #new-token: 3, #cached-token: 31, token usage: 0.00, #running-req: 1, #queue-req: 0
[2025-05-18 11:49:26] INFO:     127.0.0.1:47570 - "POST /generate HTTP/1.1" 200 OK


[2025-05-18 11:49:26] Prefill batch. #new-seq: 1, #new-token: 13, #cached-token: 33, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-05-18 11:49:26] Decode batch. #running-req: 1, #token: 84, token usage: 0.00, cuda graph: False, gen throughput (token/s): 57.22, #queue-req: 0
[2025-05-18 11:49:26] INFO:     127.0.0.1:47586 - "POST /generate HTTP/1.1" 200 OK


## Parallelism

Use `fork` to launch parallel prompts. Because `sgl.gen` is non-blocking, the for loop below issues two generation calls in parallel.

In [7]:
@function
def tip_suggestion(s):
    s += assistant(
        "Here are two tips for staying healthy: "
        "1. Balanced Diet. 2. Regular Exercise.\n\n"
    )

    forks = s.fork(2)
    for i, f in enumerate(forks):
        f += assistant(
            f"Now, expand tip {i+1} into a paragraph:\n"
            + gen("detailed_tip", max_tokens=256, stop="\n\n")
        )

    s += assistant("Tip 1:" + forks[0]["detailed_tip"] + "\n")
    s += assistant("Tip 2:" + forks[1]["detailed_tip"] + "\n")
    s += assistant(
        "To summarize the above two tips, I can say:\n" + gen("summary", max_tokens=512)
    )


state = tip_suggestion()
print_highlight(state["summary"])

[2025-05-18 11:49:26] Prefill batch. #new-seq: 1, #new-token: 35, #cached-token: 14, token usage: 0.00, #running-req: 0, #queue-req: 0
[2025-05-18 11:49:26] Prefill batch. #new-seq: 1, #new-token: 35, #cached-token: 14, token usage: 0.00, #running-req: 1, #queue-req: 0


[2025-05-18 11:49:27] Decode batch. #running-req: 2, #token: 132, token usage: 0.01, cuda graph: False, gen throughput (token/s): 89.75, #queue-req: 0


[2025-05-18 11:49:28] Decode batch. #running-req: 2, #token: 212, token usage: 0.01, cuda graph: False, gen throughput (token/s): 124.28, #queue-req: 0


[2025-05-18 11:49:28] Decode batch. #running-req: 2, #token: 292, token usage: 0.01, cuda graph: False, gen throughput (token/s): 125.34, #queue-req: 0


[2025-05-18 11:49:29] INFO:     127.0.0.1:47596 - "POST /generate HTTP/1.1" 200 OK
[2025-05-18 11:49:29] Decode batch. #running-req: 1, #token: 208, token usage: 0.01, cuda graph: False, gen throughput (token/s): 113.54, #queue-req: 0


...

[2025-05-18 11:49:30] Decode batch. #running-req: 1, #token: 455, token usage: 0.02, cuda graph: False, gen throughput (token/s): 64.76, #queue-req: 0


[2025-05-18 11:49:31] Decode batch. #running-req: 1, #token: 495, token usage: 0.02, cuda graph: False, gen throughput (token/s): 63.87, #queue-req: 0


[2025-05-18 11:49:31] Decode batch. #running-req: 1, #token: 535, token usage: 0.03, cuda graph: False, gen throughput (token/s): 63.75, #queue-req: 0


[2025-05-18 11:49:32] Decode batch. #running-req: 1, #token: 575, token usage: 0.03, cuda graph: False, gen throughput (token/s): 64.09, #queue-req: 0


[2025-05-18 11:49:33] INFO:     127.0.0.1:47602 - "POST /generate HTTP/1.1" 200 OK


## Constrained Decoding

Use `regex` to specify a regular expression as a decoding constraint. This is only supported for local models.

In [8]:
@function
def regular_expression_gen(s):
    s += user("What is the IP address of the Google DNS servers?")
    s += assistant(
        gen(
            "answer",
            temperature=0,
            regex=r"((25[0-5]|2[0-4]\d|[01]?\d\d?).){3}(25[0-5]|2[0-4]\d|[01]?\d\d?)",
        )
    )


state = regular_expression_gen()
print_highlight(state["answer"])

[2025-05-18 11:49:33] Prefill batch. #new-seq: 1, #new-token: 18, #cached-token: 12, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-05-18 11:49:33] Decode batch. #running-req: 1, #token: 40, token usage: 0.00, cuda graph: False, gen throughput (token/s): 32.42, #queue-req: 0
[2025-05-18 11:49:33] INFO:     127.0.0.1:36282 - "POST /generate HTTP/1.1" 200 OK


Use `regex` to define a `JSON` decoding schema.

In [9]:
character_regex = (
    r"""\{\n"""
    + r"""    "name": "[\w\d\s]{1,16}",\n"""
    + r"""    "house": "(Gryffindor|Slytherin|Ravenclaw|Hufflepuff)",\n"""
    + r"""    "blood status": "(Pure-blood|Half-blood|Muggle-born)",\n"""
    + r"""    "occupation": "(student|teacher|auror|ministry of magic|death eater|order of the phoenix)",\n"""
    + r"""    "wand": \{\n"""
    + r"""        "wood": "[\w\d\s]{1,16}",\n"""
    + r"""        "core": "[\w\d\s]{1,16}",\n"""
    + r"""        "length": [0-9]{1,2}\.[0-9]{0,2}\n"""
    + r"""    \},\n"""
    + r"""    "alive": "(Alive|Deceased)",\n"""
    + r"""    "patronus": "[\w\d\s]{1,16}",\n"""
    + r"""    "bogart": "[\w\d\s]{1,16}"\n"""
    + r"""\}"""
)


@function
def character_gen(s, name):
    s += user(
        f"{name} is a character in Harry Potter. Please fill in the following information about this character."
    )
    s += assistant(gen("json_output", max_tokens=256, regex=character_regex))


state = character_gen("Harry Potter")
print_highlight(state["json_output"])

[2025-05-18 11:49:34] Prefill batch. #new-seq: 1, #new-token: 24, #cached-token: 14, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-05-18 11:49:34] Decode batch. #running-req: 1, #token: 73, token usage: 0.00, cuda graph: False, gen throughput (token/s): 37.05, #queue-req: 0


[2025-05-18 11:49:35] Decode batch. #running-req: 1, #token: 113, token usage: 0.01, cuda graph: False, gen throughput (token/s): 62.76, #queue-req: 0


[2025-05-18 11:49:36] Decode batch. #running-req: 1, #token: 153, token usage: 0.01, cuda graph: False, gen throughput (token/s): 62.23, #queue-req: 0
[2025-05-18 11:49:36] INFO:     127.0.0.1:36294 - "POST /generate HTTP/1.1" 200 OK


## Batching 

Use `run_batch` to run a batch of prompts.

In [10]:
@function
def text_qa(s, question):
    s += user(question)
    s += assistant(gen("answer", stop="\n"))


states = text_qa.run_batch(
    [
        {"question": "What is the capital of the United Kingdom?"},
        {"question": "What is the capital of France?"},
        {"question": "What is the capital of Japan?"},
    ],
    progress_bar=True,
)

for i, state in enumerate(states):
    print_highlight(f"Answer {i+1}: {states[i]['answer']}")

[2025-05-18 11:49:36] Prefill batch. #new-seq: 1, #new-token: 1, #cached-token: 13, token usage: 0.00, #running-req: 0, #queue-req: 0
[2025-05-18 11:49:36] INFO:     127.0.0.1:36300 - "POST /generate HTTP/1.1" 200 OK


  0%|          | 0/3 [00:00<?, ?it/s]

[2025-05-18 11:49:36] Prefill batch. #new-seq: 1, #new-token: 9, #cached-token: 17, token usage: 0.00, #running-req: 0, #queue-req: 0
[2025-05-18 11:49:36] Prefill batch. #new-seq: 1, #new-token: 11, #cached-token: 17, token usage: 0.00, #running-req: 1, #queue-req: 0
[2025-05-18 11:49:36] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 19, token usage: 0.00, #running-req: 2, #queue-req: 0


 33%|███▎      | 1/3 [00:00<00:00,  5.36it/s]

 67%|██████▋   | 2/3 [00:00<00:00,  8.49it/s]

[2025-05-18 11:49:36] INFO:     127.0.0.1:36324 - "POST /generate HTTP/1.1" 200 OK
[2025-05-18 11:49:36] INFO:     127.0.0.1:36336 - "POST /generate HTTP/1.1" 200 OK


100%|██████████| 3/3 [00:00<00:00, 13.78it/s]

[2025-05-18 11:49:36] INFO:     127.0.0.1:36308 - "POST /generate HTTP/1.1" 200 OK


## Streaming 

Use `stream` to stream the output to the user.

In [11]:
@function
def text_qa(s, question):
    s += user(question)
    s += assistant(gen("answer", stop="\n"))


state = text_qa.run(
    question="What is the capital of France?", temperature=0.1, stream=True
)

for out in state.text_iter():
    print(out, end="", flush=True)

<|im_start|>system
You are a helpful assistant.<|im_end|>
<|im_start|>user
What is the capital of France?<|im_end|>
<|im_start|>assistant


[2025-05-18 11:49:36] INFO:     127.0.0.1:36346 - "POST /generate HTTP/1.1" 200 OK
[2025-05-18 11:49:36] Prefill batch. #new-seq: 1, #new-token: 1, #cached-token: 25, token usage: 0.00, #running-req: 0, #queue-req: 0
The

 capital

 of

 France

 is

 Paris

.

<|im_end|>


## Complex Prompts

You may use `{system|user|assistant}_{begin|end}` to define complex prompts.

In [12]:
@function
def chat_example(s):
    s += system("You are a helpful assistant.")
    # Same as: s += s.system("You are a helpful assistant.")

    with s.user():
        s += "Question: What is the capital of France?"

    s += assistant_begin()
    s += "Answer: " + gen("answer", max_tokens=100, stop="\n")
    s += assistant_end()


state = chat_example()
print_highlight(state["answer"])

[2025-05-18 11:49:36] Prefill batch. #new-seq: 1, #new-token: 17, #cached-token: 14, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-05-18 11:49:36] INFO:     127.0.0.1:36354 - "POST /generate HTTP/1.1" 200 OK


In [13]:
terminate_process(server_process)

[2025-05-18 11:49:36] Child process unexpectedly failed with an exit code 9. pid=1055799


[2025-05-18 11:49:36] Child process unexpectedly failed with an exit code 9. pid=1055733


## Multi-modal Generation

You may use SGLang frontend language to define multi-modal prompts.
See [here](https://docs.sglang.ai/supported_models/generative_models.html) for supported models.

In [14]:
server_process, port = launch_server_cmd(
    "python -m sglang.launch_server --model-path Qwen/Qwen2.5-VL-7B-Instruct --host 0.0.0.0"
)

wait_for_server(f"http://localhost:{port}")
print(f"Server started on http://localhost:{port}")

[2025-05-18 11:49:43] server_args=ServerArgs(model_path='Qwen/Qwen2.5-VL-7B-Instruct', tokenizer_path='Qwen/Qwen2.5-VL-7B-Instruct', tokenizer_mode='auto', skip_tokenizer_init=False, load_format='auto', trust_remote_code=False, dtype='auto', kv_cache_dtype='auto', quantization=None, quantization_param_path=None, context_length=None, device='cuda', served_model_name='Qwen/Qwen2.5-VL-7B-Instruct', chat_template=None, completion_template=None, is_embedding=False, enable_multimodal=None, revision=None, host='0.0.0.0', port=37541, mem_fraction_static=0.88, max_running_requests=200, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False, random_seed=356540598, constrained_json_whitespace_pattern=None, watchdog_timeout=300, dist_timeout=None, download_dir=None, base_gpu_id=0, gpu_id_step=1, log_level

[2025-05-18 11:49:46] Infer the chat template name from the model path and obtain the result: qwen2-vl.


[2025-05-18 11:49:53] Attention backend not set. Use flashinfer backend by default.
[2025-05-18 11:49:53] Automatically reduce --mem-fraction-static to 0.792 because this is a multimodal model.
[2025-05-18 11:49:53] Automatically turn off --chunked-prefill-size for multimodal model.
[2025-05-18 11:49:53] Init torch distributed begin.
[2025-05-18 11:49:53] Init torch distributed ends. mem usage=0.00 GB
[2025-05-18 11:49:53] Load weight begin. avail mem=78.60 GB


[2025-05-18 11:49:54] Multimodal attention backend not set. Use sdpa.


[2025-05-18 11:49:54] Using model weights format ['*.safetensors']


...

[2025-05-18 11:50:07] INFO:     127.0.0.1:33452 - "GET /v1/models HTTP/1.1" 200 OK


[2025-05-18 11:50:08] INFO:     127.0.0.1:33456 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-05-18 11:50:08] Prefill batch. #new-seq: 1, #new-token: 6, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0


2025-05-18 11:50:09,742 - INFO - flashinfer.jit: Loading JIT ops: batch_prefill_with_kv_cache_dtype_q_bf16_dtype_kv_bf16_dtype_o_bf16_dtype_idx_i32_head_dim_qk_128_head_dim_vo_128_posenc_0_use_swa_False_use_logits_cap_False_f16qk_False
2025-05-18 11:50:09,765 - INFO - flashinfer.jit: Finished loading JIT ops: batch_prefill_with_kv_cache_dtype_q_bf16_dtype_kv_bf16_dtype_o_bf16_dtype_idx_i32_head_dim_qk_128_head_dim_vo_128_posenc_0_use_swa_False_use_logits_cap_False_f16qk_False


[2025-05-18 11:50:10] INFO:     127.0.0.1:33460 - "POST /generate HTTP/1.1" 200 OK
[2025-05-18 11:50:10] The server is fired up and ready to roll!


Server started on http://localhost:37541


In [15]:
set_default_backend(RuntimeEndpoint(f"http://localhost:{port}"))

[2025-05-18 11:50:12] INFO:     127.0.0.1:60848 - "GET /get_model_info HTTP/1.1" 200 OK


Ask a question about an image.

In [16]:
@function
def image_qa(s, image_file, question):
    s += user(image(image_file) + question)
    s += assistant(gen("answer", max_tokens=256))


image_url = "https://github.com/sgl-project/sglang/blob/main/test/lang/example_image.png?raw=true"
image_bytes, _ = load_image(image_url)
state = image_qa(image_bytes, "What is in the image?")
print_highlight(state["answer"])

[2025-05-18 11:50:13] Prefill batch. #new-seq: 1, #new-token: 307, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-05-18 11:50:14] Decode batch. #running-req: 1, #token: 340, token usage: 0.02, cuda graph: False, gen throughput (token/s): 5.15, #queue-req: 0


[2025-05-18 11:50:15] Decode batch. #running-req: 1, #token: 380, token usage: 0.02, cuda graph: False, gen throughput (token/s): 38.45, #queue-req: 0
[2025-05-18 11:50:16] INFO:     127.0.0.1:60862 - "POST /generate HTTP/1.1" 200 OK


In [17]:
terminate_process(server_process)

[2025-05-18 11:50:16] Child process unexpectedly failed with an exit code 9. pid=1056924
[2025-05-18 11:50:16] Child process unexpectedly failed with an exit code 9. pid=1056858
